In [1]:
import numpy as np
import pandas as pd
import json
import re
import os
from google.colab import drive
import random
import torch

In [2]:
!pip install -q transformers
!pip install -q huggingface_hub
!pip install -q datasets
!pip install -q accelerate
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q trl
!pip install --upgrade -q trl

In [3]:
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
model_id="TinyLlama/TinyLlama_v1.1_math_code"

In [6]:
# Load your dataset (example with input-output pairs)
from datasets import load_dataset
from transformers import AutoTokenizer

train_dataset = load_dataset('csv', data_files=['/content/drive/MyDrive/Fine-tuning-llms/qe_input_dataset.csv'],\
                       column_names=['input','output'], split='train[:80%]')
eval_dataset = load_dataset('csv', data_files=['/content/drive/MyDrive/Fine-tuning-llms/qe_input_dataset.csv'],\
                       column_names=['input','output'], split='train[80%:]')

# Load the tokenizer for your model (LLaMA tokenizer, for example)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Define a function to tokenize your dataset
def preprocess_function(examples):
    texts=[]
    for inputs,outputs in zip(examples['input'],examples['output']):
        input_text = 'user: '+ inputs + 'assistant: '+ outputs + tokenizer.eos_token
        texts.append(input_text)

    tokenized_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=2048,return_tensors = "pt")
    return tokenized_inputs

# Apply the preprocessing function to your dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Remove the original columns (instruction, output) as we now have input_ids and labels
train_dataset = train_dataset.remove_columns(['input', 'output'])
eval_dataset = eval_dataset.remove_columns(['input', 'output'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [8]:
from transformers import AutoModelForCausalLM, set_seed

base_model = AutoModelForCausalLM.from_pretrained(
        "TinyLlama/TinyLlama_v1.1_math_code",
        device_map = 'auto')


base_model.config.use_cache = False
set_seed(123)

In [9]:
from peft import LoraConfig

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.01,
    task_type="CAUSAL_LM",
)

In [14]:
from peft import prepare_model_for_kbit_training, get_peft_model
from transformers import TrainingArguments
from transformers import BitsAndBytesConfig
import torch

# # bitsandbytes
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
device_map = "auto"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(model, peft_config)

train_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Fine-tuning-llms/test_output',
    evaluation_strategy="epoch",
    max_steps = 60,
    weight_decay=0,
    save_strategy='epoch',
    save_steps=20,
    warmup_ratio=0.09,
    learning_rate=0.002,
    max_grad_norm=0.3,
    gradient_checkpointing=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=3,
    per_device_eval_batch_size=8,
    # num_train_epochs=1,
    logging_steps=2,
    fp16=True,
    load_best_model_at_end=True,

)
from transformers import Trainer
trainer = Trainer(
        model=model,
        args=train_args,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        # eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

torch.float16


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [15]:
def print_trainable_parameters(args, model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
    )

# Verifying the datatypes and parameter counts before training.
print_trainable_parameters(train_args, model)
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total+= v
for k, v in dtypes.items():
    print(k, v, v/total)

trainable params: 13160448 || all params: 1113208832 || 
torch.float32 1113208832 1.0


In [16]:
13160448/1113208832

0.011822083711243858

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Fine-tuning-llms/test_output/model_tiny/")
peft_config.save_pretrained("/content/drive/MyDrive/Fine-tuning-llms/test_output/model_tiny/")